# word2vec

Мы уже работали с грамматикой — в автоматическом кодировании грамматических категорий, кажется, ничего сверхъестественного нет. Но можно ли как-то закодировать значение слова? Да! В этом нам поможет лингвистическая теория, которая называется дистрибутивной гипотезой. Она утверждает, что значение слова определяется его контекстом — иначе говоря, словами, которые встречаются рядом с этим словом в тексте. Область лингвистики, которая занимается вычислением степени семантической близости между словами/текстами и т.п. на основании их распределения (дистрибуции) в больших массивах данных (текстовых корпусах) назвается **дистрибутивной семантикой**.

Одной из самых известных моделей для работы с дистрибутиыной семантикой является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations inVector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)

Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.

### Зачем это нужно?

* Решать лингвистические задачи (в основном это про семантику и сочетаемость)
* Подавать на вход нейронным сетям

### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


<img src="https://nycdatascience.com/blog/wp-content/uploads/2017/06/cossim.png" width="500">


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)



# Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html). Вообще-то gensim — библиотека для тематического моделирования текстов, но один из компонентов в ней — реализация на python алгоритмов из библиотеки word2vec (которая в оригинале была написана на C++).

Если gensim у вас не стоит, то ставим: `pip install gensim`. Можно сделать это прямо из jupyter'а! Чтобы выполнить какую-то команду не в питоне,  в командной строке, нужно написать перед ней восклицательный знак. 


In [3]:
! pip install gensim

In [5]:
import re
import gensim
import logging
import pandas as pd
import urllib.request
from gensim.models import word2vec

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Как обучить свою модель

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

Поскольку иногда тренировка модели занимает много времени, то можно ещё вести лог событий, чтобы понимать, что на каком этапе происходит.

In [6]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
# ЭТУ ЯЧЕЙКУ ЗАПУСКАТЬ НЕ НАДО

from tqdm import tqdm
from pymystem3 import Mystem

m = Mystem()
sw = stopwords.words('russian')

with open('liza.txt', 'r', encoding='utf8') as f:
    text = f.readlines()

new_lines = []

for line in tqdm(text):
    line = ' '.join([w for w in line.split() if w not in sw])
    newline = ''.join(m.lemmatize(line))
    new_lines.append(newline)
        
with open('liza_lem.txt', 'w', encoding='utf8') as f1:
    for line in new_lines:
        f1.write(line)

100%|████████████████████████████████████████████████████████████| 395/395 [08:18<00:00,  1.26s/it]


На вход модели даем текстовый файл, каждое предложение на отдельной строчке. Вот игрушечный пример с текстом «Бедной Лизы». Он заранее очищен от пунктуации, приведен к нижнему регистру и лемматизирован.

In [7]:
f = 'liza_lem.txt'
data = gensim.models.word2vec.LineSentence(f)

Инициализируем модель. Основные параметры:

* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [8]:
%time model_liza = gensim.models.Word2Vec(data, size=300, window=5, min_count=2, workers=2)

D:\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:323: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
2019-04-27 13:27:01,465 : INFO : collecting all words and their counts
2019-04-27 13:27:01,500 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-27 13:27:01,507 : INFO : collected 1213 word types from a corpus of 3109 raw words and 392 sentences
2019-04-27 13:27:01,508 : INFO : Loading a fresh vocabulary
2019-04-27 13:27:01,513 : INFO : min_count=2 retains 478 unique words (39% of original 1213, drops 735)
2019-04-27 13:27:01,514 : INFO : min_count=2 leaves 2374 word corpus (76% of original 3109, drops 735)
2019-04-27 13:27:01,521 : INFO : deleting the raw counts dictionary of 1213 items
2019-04-27 13:27:01,523 : INFO : sample=0.001 downsamples 83 most-common words
2019-04-27 13:27:01,527 : INFO : downsampling leaves e

Wall time: 3.73 s


Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

In [8]:
model_liza.init_sims(replace=True)
model_path = "liza.bin"

print("Saving model...")
model_liza.wv.save_word2vec_format(model_path, binary=True)

2019-04-14 20:56:47,078 : INFO : precomputing L2-norms of word weight vectors


Saving model...


2019-04-14 20:56:47,090 : INFO : storing 478x300 projection weights into liza.bin


Смотрим, сколько в модели слов:

In [9]:
print(len(model_liza.wv.vocab))

478


In [15]:
print(sorted([w for w in model_liza.wv.vocab]))

['анюта', 'армия', 'ах', 'барин', 'бедный', 'белый', 'берег', 'березовый', 'беречь', 'бесчисленный', 'благодарить', 'бледный', 'блеснуть', 'блестящий', 'близ', 'бог', 'богатый', 'большой', 'бояться', 'брать', 'бросать', 'бросаться', 'бывать', 'быть', 'важный', 'ввечеру', 'вдова', 'велеть', 'великий', 'великолепный', 'верить', 'верно', 'весело', 'веселый', 'весна', 'вести', 'весь', 'весьма', 'ветвь', 'ветер', 'вечер', 'взглядывать', 'вздох', 'вздыхать', 'взор', 'взять', 'вид', 'видеть', 'видеться', 'видный', 'вместе', 'вода', 'возвращаться', 'воздух', 'война', 'воображать', 'воображение', 'воспоминание', 'восторг', 'восхищаться', 'время', 'все', 'вслед', 'вставать', 'встречаться', 'всякий', 'высокий', 'выть', 'выходить', 'глаз', 'глубокий', 'гнать', 'говорить', 'год', 'голос', 'гора', 'горе', 'горестный', 'горлица', 'город', 'горький', 'господь', 'гром', 'грусть', 'давать', 'давно', 'далее', 'дверь', 'движение', 'двор', 'девушка', 'дело', 'день', 'деньги', 'деревня', 'деревянный', 'деся

И чему же мы ее научили? Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [25]:
print(model_liza.wv.most_similar(positive=["смерть", "любовь"], negative=["печальный"], topn=1))

print(model_liza.wv.most_similar("любовь", topn=3))

print(model_liza.wv.similarity("лиза", "эраст"))

print(model_liza.wv.doesnt_match("скорбь грусть слеза улыбка".split()))

[('приятный', 0.19232240319252014)]
[('смерть', 0.17755034565925598), ('сей', 0.17514510452747345), ('дело', 0.17496509850025177)]
0.17681161637929305
грусть


## Как использовать готовую модель

### RusVectōrēs

На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».

Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/)


### Работа с моделью

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же гласса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [2]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x26412929ac8>)

In [26]:
m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

2019-04-14 21:05:06,709 : INFO : loading projection weights from ruscorpora_mystem_cbow_300_2_2015.bin.gz
2019-04-14 21:05:25,124 : INFO : loaded (281776, 300) matrix from ruscorpora_mystem_cbow_300_2_2015.bin.gz


In [27]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'студент_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [28]:
for word in words:
    # есть ли слово в модели? 
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]


2019-04-14 21:06:30,278 : INFO : precomputing L2-norms of word weight vectors


неделя_S 0.7165195941925049
месяц_S 0.631048858165741
вечер_S 0.5828739404678345
утро_S 0.5676207542419434
час_S 0.5605547428131104
минута_S 0.5297019481658936
гекатомбеон_S 0.4897990822792053
денек_S 0.48224714398384094
полчаса_S 0.48217129707336426
ночь_S 0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S 0.6946247816085815
утро_S 0.57301926612854
ноченька_S 0.5582467317581177
рассвет_S 0.5553582906723022
ночка_S 0.5351512432098389
полдень_S 0.5334426164627075
полночь_S 0.478694349527359
день_S 0.4780748784542084
сумерки_S 0.4390218257904053
фундерфун_S 0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681  0.04785006]
женщина_S 0.5979775190353394
парень_S 0.4991787374019623
мужчина_S 0.4767409563064575
мужик_S 0.47384002804756165
россиянин_S 0.47190436720848083
народ_S 0.4654741883277893
согр

Находим косинусную близость пары слов:

In [29]:
print(model.similarity('человек_S', 'обезьяна_S'))

0.23895609343220617


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [30]:
print(model.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


Найди лишнее!

In [34]:
print(model.doesnt_match('мир_S жизнь_S бытие_S ничто_S'.split()))

2019-04-14 21:07:22,814 : WARNING : vectors for words {'ничто_S'} are not present in the model, ignoring these words


мир_S


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния о слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификатор мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [35]:
res = model.accuracy('ru_analogy_tagged.txt')

2019-04-14 21:10:00,590 : INFO : capital-common-countries: 19.0% (58/306)
2019-04-14 21:10:02,893 : INFO : capital-world: 10.1% (52/515)
2019-04-14 21:10:03,481 : INFO : currency: 4.6% (6/130)
2019-04-14 21:10:04,894 : INFO : family: 71.2% (218/306)
2019-04-14 21:10:08,514 : INFO : gram1-Aective-to-adverb: 18.7% (152/812)
2019-04-14 21:10:10,428 : INFO : gram2-opposite: 32.1% (122/380)
2019-04-14 21:10:14,694 : INFO : gram6-nationality-Aective: 32.3% (293/907)
2019-04-14 21:10:14,696 : INFO : total: 26.8% (901/3356)


In [36]:
print(res[4]['incorrect'][:10])

[('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ДЕД_S', 'БАБКА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'КОРОЛЬ_S', 'КОРОЛЕВА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПРИНЦ_S', 'ПРИНЦЕССА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ДЕД_S', 'БАБКА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('ПАПА_S', 'МАМА_S', 'ДЕД_S', 'БАБКА_S'), ('ПАПА_S', 'МАМА_S', 'ОТЧИМ_S', 'МАЧЕХА_S')]


# Задание

1. Скачайте корпус региональной прессы [отсюда](https://www.dropbox.com/s/46mn8dp3l3jgkx0/regional_papers.rar?dl=0). Да-да, это те самые данные, которые вы собрали в прошлом семестре!
2. Лемматизируйте корпус, очистите от пунктуации и служебной информации и обучите на нем модель word2vec. Для начала можно взять какую-нибудь одну газету.
3. Найдите по 5 ближайших слов к словам "город", "деревня", "спорт", "бизнес", "Россия", "происшествие", "река", "озеро", "море", "горы", "депутат", "врач", "север", "юг", "Кавказ", "Сибирь", "газпром", название своего родного города... Учтите, что слова может не быть в модели!
4. Посчитайте семантическую близкость слов "театр" и "кино", "Владивосток" и "Москва", "церковь" и "государство", "культура" и "отдых", "преступление" и "наказание". Можете придумать свои пары слов.
3. Решите примеры (можно придумать свои):
        * москва + екатеринбург - собянин
        * спартак - москва + санкт-петербург
        * иркутск - байкал + сочи
        * татарстан - татарский + бурятия
        * чай - лимон + кофе
        * авиакомпания - аэрофлот + ржд
4. Найдите лишнее, попробуйте интерпретировать результаты
        * магазин, супермаркет, рынок, тц
        * теннис, хоккей, футбол, дзюдо
        * кошка, собака, попугай, кролик
        * коми, дагестан, башкирия, камчатка

Сохраните модель, она вам еще понадобится!